# Query Million Song Dataset

If you have not already done so, make sure to run `cache_msd.ipynb` to cache msd to `msd.pickle` before running this notebook.

## Imports

In [10]:
import dill as pickle
import numpy as np
import time, numbers, os, pdb
import shutil

## Utils

In [23]:
def query_dict_array(arr, query):
    
    if type(query) != list:
        query = [query]
    
    def array_filter(data):
        for q in query:
            t = 'OR'
            if 'type' in q:
                if q['type'].upper() == 'AND':
                    t = 'AND'    
            for key in q:
                
                if key == 'type':
                    continue
                
                if t == 'OR':
                    if type(q[key]) == str or isinstance(q[key], numbers.Number):
                        if type(data[key]) == str or isinstance(data[key], numbers.Number):
                            if q[key] == data[key]:
                                return True
                        elif type(data[key]) == list or type(data[key]) == np.ndarray:
                            if q[key] in data[key]:
                                return True
                    elif type (q[key]) == list:
                        for v in q[key]:
                            if type(data[key]) == str or isinstance(v, numbers.Number):
                                if v == data[key]:
                                    return True
                            elif type(data[key]) == list or type(data[key]) == np.ndarray:
                                if v in data[key]:
                                    return True
                    elif callable(q[key]):
                        if q[key](data[key]):
                            return True
                else:
                    if type(q[key]) == str or isinstance(q[key], numbers.Number):
                        if type(data[key]) == str or isinstance(data[key], numbers.Number):
                            if q[key] != data[key]:
                                return False
                        elif type(data[key]) == list or type(data[key]) == np.ndarray:
                            if q[key] not in data[key]:
                                return False
                    elif type (q[key]) == list:
                        for v in q[key]:
                            if type(data[key]) == str or isinstance(v, numbers.Number):
                                if v != data[key]:
                                    return False
                            elif type(data[key]) == list or type(data[key]) == np.ndarray:
                                if v not in data[key]:
                                    return False
                    elif callable(q[key]):
                        if not q[key](data[key]):
                            return False
            if t == 'AND':
                return True
        return False

    return filter(array_filter, arr)

def create_symlink_dir(paths, dirname, file_limit_per_track=None):
    '''Takes a list of msd paths and creates a new directory 
       dirname with symbolic links to the files pointed to by paths.'''
    
    # delete dirname if it exists
    if os.path.isdir(dirname):
        shutil.rmtree(dirname)
    
    # create dirname if it doesn't exist
    if not os.path.isdir(dirname):
        os.mkdir(dirname)
        
    dupes = 0
    for path in paths:
        if os.path.exists(path):
            for i, filename in enumerate(os.listdir(path)):
                if file_limit_per_track != None and i == file_limit_per_track:
                    break
                if os.path.exists(os.path.join(dirname, filename)):
                    dupes = dupes + 1
                    continue
                os.symlink(os.path.join(path, filename), os.path.join(dirname, filename))
    print('found {} duplicate midi files'.format(dupes))

## Load MSD Cache

In [3]:
start_time = time.time()
with open('msd.pickle', 'r') as f:
    msd = pickle.load(f)
print('Loaded in {:.2f} seconds'.format(time.time() - start_time))

Loaded in 9.96 seconds


## Query dataset

Below are some examples of AND and OR million song dataset queries that can be made against the cached msd dataset.

In [33]:
## All songs by artists w/ terms 'jazz' AND a hotttnesss greater than 0.5
query = {
    'song_hotttnesss': lambda hotttnesss: hotttnesss > 0.6,
    'artist_terms': ['electronic'],
    'song_year': lambda year: year > 1990,
    'type': 'AND'
}

results = query_dict_array(msd, query)

14304


Once the query is complete, lets create a new directory and populate it with symlinks to the files pointed to by the results.

In [32]:
msd_path = '/home/bbpwn2/Documents/code/midi-dataset/data/lmd_matched/{}'
symlink_dir = 'test'
tracks = [msd_path.format(r['path']) for r in results]
create_symlink_dir(tracks, symlink_dir, file_limit_per_track=1)
print('symlinks created in directory: {}'.format(symlink_dir))

found 4420 duplicate midi files
symlinks created in directory: test
